<a href="https://colab.research.google.com/github/PatrickRaborn/DS-Unit-2-Kaggle-Challenge/blob/master/Copy_of_DS_module_project_221.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 2, Module 1*

---

In [1]:
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    !pip install category_encoders==2.*
    !pip install pandas-profiling==2.*

DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'

     |████████████████████████████████| 81kB 5.1MB/s 
     |████████████████████████████████| 245kB 7.6MB/s 
     |████████████████████████████████| 81kB 8.0MB/s 
     |████████████████████████████████| 604kB 11.6MB/s 
     |████████████████████████████████| 61kB 8.6MB/s 
     |████████████████████████████████| 3.1MB 16.2MB/s 
     |████████████████████████████████| 81kB 11.1MB/s 
     |████████████████████████████████| 296kB 50.5MB/s 
  Created wheel for htmlmin: filename=htmlmin-0.1.12-cp36-none-any.whl size=27085 sha256=849038697a85b9cc5d9dc3eff5295131c724cf85b3008943348929dd02e624a8
  Stored in directory: /root/.cache/pip/wheels/43/07/ac/7c5a9d708d65247ac1f94066cf1db075540b85716c30255459
  Created wheel for phik: filename=phik-0.11.0-cp36-none-any.whl size=599738 sha256=b9befeba27b2c1b65e7e8391e2c418217148de0cf788bdbf8bf2558438afe053
  Stored in directory: /root/.cache/pip/wheels/af/54/11/aba77f21075918de02f7964eabfe8c10d5542df9e6ad10b225
Successfully built htmlmin phik
ERROR: goog

# Decision Trees

## Kaggle

**Task 1:** [Sign up for a Kaggle account](https://www.kaggle.com/), if you don’t already have one. Go to our Kaggle InClass competition website (the URL is in Slack). Go to the Rules page. Accept the rules of the competition. Notice that the Rules page also has instructions for the Submission process. The Data page has feature definitions.

## Wrangle Data

**Task 2:** Add to the code below so that `id` is set as the index for `df`.

In [2]:
# df.head()

In [65]:
import pandas as pd


# YOUR CODE HERE
def wrangle(df):
# make a copy
  df = df.copy()
# drop repeated rows (observations)
  #df.drop_duplicates(inplace=True)
# drop constant features
  df.drop(columns=['quantity_group','extraction_type_group'],inplace=True)
#drop constant features
  df.drop(columns=['recorded_by'],inplace=True)
# drop features with lots of NaN values
  df.dropna(axis=1,thresh=7_000, inplace=True)
# high cardinality 
  drop_cols = [col for col in df.select_dtypes('object').columns
             if df[col].nunique() > 100]
  df.drop(columns=drop_cols,inplace=True)
# drop datetime column
  df.drop(columns='date_recorded')
# feature engineering
  df['pump_age'] = df['date_recorded'].dt.year - df['construction_year']
  df['date_recorded'] = df['date_recorded'].dt.year
  return df
#df.head()
# import data
df = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv',
                          parse_dates=['date_recorded'],
                          na_values=[0, -2.000000e-08]),
              pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv')).set_index('id')

In [66]:
df = wrangle(df)

In [67]:
df.shape

(59399, 30)

**Test 2**

In [42]:
'''Task 2 Testing'''

assert isinstance(df, pd.DataFrame), 'Have you created the DataFrame `df`?'
assert df.shape == (59400, 40), '`df` is the wrong shape. Did you set the index to `id`?'
assert 69572 in df.index, 'The index for `df` is has the wrong values. Did you set the index to `id`?'

AssertionError: ignored

## Split Data

**Task 3:** Create your target vector `y` and feature matrix `X`.

In [68]:
target = 'status_group'

# YOUR CODE HERE
X = df.drop(columns=target)
y = df[target]

**Test 3**

In [7]:
'''Task 3 Testing'''

assert isinstance(X, pd.DataFrame), '`X` is the wrong data type.'
assert isinstance(y, pd.Series), '`y` is the wrong data type.'
assert y.shape == (59400,), '`y` is the wrong shape.'
assert X.shape == (59400,39), '`X` is the wrong shape.'

We already have a test set for this model, the `test.csv` that you'll use to make the predictions you'll upload to Kaggle. However, since our competition only allows for 2 submissions per day, we need a way to estimate our training error so that we can quickly iterate and improve our model. We can achieve this by creating a validation set from the data we have.

**Task 4:** Split `X` and `y` into training and validation sets. Your validation set should be 20% of the data you have. You should have four variables: `X_train`, `X_val`, `y_train`, and `y_val`. 

In [69]:
from sklearn.model_selection import train_test_split

# YOUR CODE HERE
X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=.8, random_state=42)

In [9]:
X_train.shape

(47520, 39)

**Test 4**

In [10]:
'''Task 4 Testing'''

assert X_train.shape == (47520, 39), '`X_train` is the wrong shape.'
assert X_val.shape == (11880, 39), '`X_val` is the wrong shape.'
assert y_train.shape == (47520,), '`y_train` is the wrong shape.'
assert y_val.shape == (11880,), '`y_val` is the wrong shape.'

## Establish Baseline

**Task 5:** This is a **classification** problem, so you need to establish the baseline accuracy for your training set. Find the majority class for `y_train` and calculate the percentage of labels in `y_train` belonging to that class. Assign your answer to the variable name `baseline_acc`.

In [70]:
# YOUR CODE HERE
baseline_acc = y_train.value_counts(normalize=True).max()

print('Baseline Accuracy:', baseline_acc)

Baseline Accuracy: 0.5425829668132747


**Task 5**

In [12]:
'''Task 5 Testing'''

assert isinstance(baseline_acc, float), '`baseline_acc` should be a `float`.'
assert 0.0 <= baseline_acc <= 1.0, '`baseline_acc` is a score that should be between 0 and 1.'

## Build Model

**Task 6:** Create a model named `model` and train it with your training data. Your model should be a pipeline with (a) transformers that you think are appropriate to this dataset and (b) a `DecisionTreeClassifier` as your predictor. **Tip:** How can you transform categorical features and missing values in order to train your model?

In [71]:
from category_encoders import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier

# YOUR CODE HERE
model = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(strategy='mean'),
    DecisionTreeClassifier()
)

model.fit(X_train, y_train);

**Test 6**

In [14]:
'''Task 6 Testing'''

assert len(model) > 1, 'Your model pipeline should have multiple steps.'
assert isinstance(model[-1], DecisionTreeClassifier), 'Your pipline should end in a `DecisionTreeClassifier`.'
assert hasattr(model, 'classes_'), 'Have you fit your model?'

## Check Metrics

**Task 7:** Calculate the training and validation accuracy of your model, and assign them to the valiables `training_acc` and `validation_acc`, respectively. Your validation accuracy should be greater than your baseline accuracy.

In [72]:
# YOUR CODE HERE
training_acc = model.score(X_train,y_train)
validation_acc = model.score(X_val,y_val)

print('Training Accuracy:', training_acc)
print('Validation Accuracy:', validation_acc)

Training Accuracy: 0.9956859361518551
Validation Accuracy: 0.7456228956228956


**Test 7**

In [16]:
'''Task 7 Testing'''

assert isinstance(training_acc, float)
assert isinstance(validation_acc, float)
assert 0.0 <= training_acc <= 1.0
assert 0.0 <= validation_acc <= 1.0

## Create Kaggle Submission

**Task 8:** Load `'waterpumps/test.csv'` into a DataFrame named `X_test`. Generate a list of predictions, and then put them into a DataFrame `submission`. Be sure that `submission` has the same index as `X_test` and that the column name for your predictions is `'status_group'`.  

In [75]:
X_test = pd.read_csv(DATA_PATH + 'waterpumps/test_features.csv',
                     parse_dates=['date_recorded'],
                     na_values=[0, -2.000000e-08])
X_test = wrangle(X_test)
# YOUR CODE HERE
# make predictions
y_pred = model.predict(X_test)
y_pred
submission = pd.DataFrame({'Status Group':y_pred}, index=X_test.index)
#submission.head()

**Test 8**

In [35]:
'''Task 8 Testing'''

assert isinstance(submission, pd.DataFrame), '`submission` should be a DataFrame.'
assert len(submission) == 14358, '`submission` should have 14358 rows.'

AssertionError: ignored

**Task 9 (`stretch goal`):** Save `submission` as a csv file using [`.to_csv()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_csv.html) and submit it to our Kaggle competition.

In [76]:
# YOUR CODE HERE
submission = pd.DataFrame({'Status Group':y_pred}, index=X_test.index).to_csv('Submission.csv')

## Explain

**Task 10 (`stretch goal`):** Make a horizontal barchart of the 10 most important features for your model.

In [ ]:
import matplotlib.pyplot as plt

# YOUR CODE HERE
raise NotImplementedError()